In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt

In [2]:
# Load astrocyte cellids into a list
astro_list = [864691134964446239, 864691135012524790, 864691135256138671, 864691135348239831, 864691135880405261, 864691136023933241]
neuro_list = [864691135269913253, 864691135337851366, 864691135341421745, 864691135386363265, 864691135497750291, 864691135657783170, 864691135837182867, 864691136194008022]
vasc_list = [864691136330394007]

print(f'number of astros:', len(astro_list))
print(f'number of neurons:', len(neuro_list))
print(f'number of vascular:', len(vasc_list))

number of astros: 6
number of neurons: 8
number of vascular: 1


In [3]:
mesh_dir = 'data/cell_meshes/dec/' # or change to your desired folder
seg_source = "precomputed://gs://h01-release/data/20210601/c3"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [4]:
# make a dictionary to hold the mesh file for each cell id in the list

mesh_dictionary = {}
for cellid in cell_list:
    mesh_file = os.path.join(mesh_dir, str(cellid) + '_0.ply')
    mesh_dictionary['cell_' + str(cellid)] = trimesh.load_mesh(mesh_file)

In [5]:
len(mesh_dictionary)

29

In [6]:
cell_opac = 0.5

# Creating mesh actors with opacity and random colors
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        random_color = list(np.random.random(size=3))
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(mesh_dictionary[cell_key], opacity=cell_opac, color=random_color)

locals().update(mesh_actor)

# Creating a camera object and defining the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000016986019770) at 0x000001698A429D08>

# gradient rendering

## color gradient linear

In [7]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

cell_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, gradient='linear'):
    if gradient == 'linear':
        opacities = 1 - (distances / max_distance)
    elif gradient == 'exponential':
        opacities = np.exp(-distances / max_distance)
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, gradient='linear')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the opacities
        opacity_array = vtk.vtkFloatArray()
        opacity_array.SetName("Opacity")
        for opacity in opacities:
            opacity_array.InsertNextValue(opacity)
        
        # Add the opacity array to the vtkPolyData
        poly_data.GetPointData().SetScalars(opacity_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.ScalarVisibilityOn()
        mapper.SetScalarModeToUsePointFieldData()
        mapper.SelectColorArray("Opacity")
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001698601B0B0) at 0x00000169FE52AB28>

## color gradient exponential

In [8]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

cell_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, gradient='linear'):
    if gradient == 'linear':
        opacities = 1 - (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, gradient='exponential')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the opacities
        opacity_array = vtk.vtkFloatArray()
        opacity_array.SetName("Opacity")
        for opacity in opacities:
            opacity_array.InsertNextValue(opacity)
        
        # Add the opacity array to the vtkPolyData
        poly_data.GetPointData().SetScalars(opacity_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.ScalarVisibilityOn()
        mapper.SetScalarModeToUsePointFieldData()
        mapper.SelectColorArray("Opacity")
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000016A33D34BE0) at 0x00000169895DBC48>

## opacity gradient

In [9]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

# cell_opac = 0.5
max_opac = 0.1
min_opac = 0.05

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (1 - (distances / max_distance))
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear')
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkFloatArray to hold the distances (to be used as scalars)
        distance_array = vtk.vtkFloatArray()
        distance_array.SetName("Distance")
        for distance in distances:
            distance_array.InsertNextValue(distance)
        
        # Add the distance array to the vtkPolyData
        poly_data.GetPointData().SetScalars(distance_array)
        
        # Create a lookup table to map distances to opacities
        lookup_table = vtk.vtkLookupTable()
        lookup_table.SetTableRange(0, max_distance)
        lookup_table.SetNumberOfTableValues(256)
        lookup_table.Build()
        
        for i in range(256):
            value = i / 255.0
            opacity = min_opac + (max_opac - min_opac) * (1 - value)
            color = [1.0, 0.0, 0.0, opacity]  # You can adjust the color as needed
            lookup_table.SetTableValue(i, color)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        mapper.SetLookupTable(lookup_table)
        mapper.UseLookupTableScalarRangeOn()
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        # Set random color (this will be overridden by the lookup table)
        random_color = list(np.random.random(size=3))
        actor.GetProperty().SetColor(random_color)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000016A33D35230) at 0x00000169FE52AA68>

## gradient opacity and colormap 

In [10]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
# import matplotlib.pyplot as plt

max_opac = 1
min_opac = 0.05

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (1 - (distances / max_distance))
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * np.exp(-k * (distances / max_distance))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.hot_r # add _r to reverse #spring #viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000016A33D32C50) at 0x000001698A44A3A8>

## reverse opacity gradient

In [11]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
# import matplotlib.pyplot as plt

max_opac = 1
min_opac = 0.000000000005

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='exponential')
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000016A33D31310) at 0x000001698A429D68>

# refine opacity gradient

In [13]:
# import vtk
# import numpy as np
# from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
# import matplotlib.pyplot as plt

max_opac = 1
min_opac = 0.5

# Define a function to calculate the opacity based on distance
def calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear'):
    if gradient == 'linear':
        opacities = min_opac + (max_opac - min_opac) * (distances / max_distance)
    elif gradient == 'exponential':
        k = 5  # You can adjust this constant to control the rate of decay
        opacities = min_opac + (max_opac - min_opac) * (1 - np.exp(-k * (distances / max_distance)))
    else:
        raise ValueError("Unsupported gradient type. Use 'linear' or 'exponential'.")
    return opacities

# Create mesh actors with gradient opacity and color
mesh_actor = {}
for cellid in cell_list:
    cell_key = 'cell_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh = mesh_dictionary[cell_key]
        centroid = mesh.centroid
        distances = np.linalg.norm(mesh.vertices - centroid, axis=1)
        max_distance = np.max(distances)
        opacities = calculate_opacity(distances, max_distance, max_opac, min_opac, gradient='linear')
        
        # Debug print to check opacity values
        print(f"Opacities (min: {np.min(opacities)}, max: {np.max(opacities)})")
        
        # Normalize distances for color mapping
        normalized_distances = distances / max_distance
        
        # Use a colormap from matplotlib
        colormap = plt.cm.hot_r #viridis
        colors = colormap(normalized_distances)
        
        # Create a vtkPoints object
        points = vtk.vtkPoints()
        for vertex in mesh.vertices:
            points.InsertNextPoint(vertex[0], vertex[1], vertex[2])
        
        # Create a vtkCellArray object
        cells = vtk.vtkCellArray()
        for face in mesh.faces:
            cells.InsertNextCell(3, face)
        
        # Create a vtkPolyData object
        poly_data = vtk.vtkPolyData()
        poly_data.SetPoints(points)
        poly_data.SetPolys(cells)
        
        # Create a vtkUnsignedCharArray to hold the colors
        color_array = vtk.vtkUnsignedCharArray()
        color_array.SetNumberOfComponents(4)  # RGBA
        color_array.SetName("Colors")
        
        for color, opacity in zip(colors, opacities):
            rgba = [int(255 * c) for c in color[:3]] + [int(255 * opacity)]
            color_array.InsertNextTuple(rgba)
        
        # Add the color array to the vtkPolyData
        poly_data.GetPointData().SetScalars(color_array)
        
        # Create a vtkPolyDataMapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(poly_data)
        
        # Create a vtkActor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        mesh_actor[cell_key] = actor

# Create a camera object and define the view
if 'cell_' + str(cell_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_' + str(cell_list[0])].centroid, backoff=400)
else:
    print(f"First cell key cell_{cell_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


Opacities (min: 0.524586629026758, max: 1.0)
Opacities (min: 0.516248373660448, max: 1.0)
Opacities (min: 0.5036108074444867, max: 1.0)
Opacities (min: 0.5114889924601367, max: 1.0)
Opacities (min: 0.5308192614331432, max: 1.0)
Opacities (min: 0.5172758568303778, max: 1.0)
Opacities (min: 0.5313134379922614, max: 1.0)
Opacities (min: 0.5005168345461923, max: 1.0)
Opacities (min: 0.5034388770990426, max: 1.0)
Opacities (min: 0.5074303618704453, max: 1.0)
Opacities (min: 0.5171018734914807, max: 1.0)
Opacities (min: 0.5043006733328208, max: 1.0)
Opacities (min: 0.5108688063184069, max: 1.0)
Opacities (min: 0.5101480620929012, max: 1.0)
Opacities (min: 0.5154998587241232, max: 1.0)
Opacities (min: 0.518411082022346, max: 1.0)
Opacities (min: 0.524503997335031, max: 1.0)
Opacities (min: 0.5186049094985198, max: 1.0)
Opacities (min: 0.5349747349856234, max: 1.0)
Opacities (min: 0.5036213875980444, max: 1.0)
Opacities (min: 0.5074024955834612, max: 1.0)
Opacities (min: 0.5011931682960111, ma

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000016A33D2DA40) at 0x000001698A44A4C8>